<a href="https://colab.research.google.com/github/lvlanson/data_mining/blob/main/Praktikum_3_Aufgabe_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
### a) Was hat einen entscheidenden Einfluss auf die Klassenzuordnung?

# Position des Objektes unbekannter Klassenzugehörigkeit
# Verteilung der Objekte der Klassen
# k Wert <=> Radius

In [2]:
# Eulersche Distanz für Berechnung der Abstände der Punkte

from math import sqrt

def euler_distance(a: tuple, b: tuple):
  sum = 0
  for x,y in zip(a,b):
    sum += (x-y)**2
  return sqrt(sum)


In [3]:
# Manhattan Distanz für Berechnung der Abstände der Punkte

def manhattan_distance(a: tuple, b:tuple):
  sum = 0
  for x,y in zip(a,b):
    sum += abs(x-y)
  return sum

In [4]:
# Funktion für Distanzberechnung

def sort_dist(a: tuple):
  return a[1]

def get_k_nearest_dist(k: int, data: list, pt: tuple, dist_func):
  if k > 0:
    k_dist = []
    for p in data:
      k_dist.append((p, dist_func(pt, p)))

    k_dist = sorted(k_dist, key=sort_dist)

    return k_dist[k-1][1]
  return 0

In [5]:
# Funktion zur Rückgabe der k-nächsten Punkte unter Klasse

def sort_nearest(a: dict):
  return list(a.values())[0]

def sort_klass(a:dict):
  return list(a.values())[0][1]

def get_k_nearest(k: int, data_dict: dict, pt: tuple, dist_func):
  # Alle gesammelten Distanzen aus Dictionaries {Klasse: Distanz}
  values = []
  for array in data_dict.items():
    # Array aus Dictionaries {Punkt: Distanz}
    distances = []
    # Array aus Dictionaries {Klasse: (Punkt, Distanz)}
    dist_class = []

    # jedem Punkt eine Distanz zuordnen
    for point in array[1]:
      distances.append({point: dist_func(pt, point)})

    # Distanzen sortieren
    distances = sorted(distances, key=sort_nearest)

    # Punkt entfernen und Klasse einsetzen
    for item in distances:
      dist_class.append({array[0]: list(item.items())[0]})
    
    values = values + dist_class
  values = sorted(values, key=sort_klass)

  # 
  return_values = {}
  max_dist = 0
  i = 0

  while i < len(values):  
    key = list(values[i].items())[0][0]
    value = list(values[i].items())[0][1]
    if i == k-1:
      max_dist = value[1]
    elif max_dist != 0  and value[1] > max_dist:
      break
     
    if return_values.get(key) is None:
      return_values[key] = [value]
    else:
      return_values[key].append(value) 
    
    i = i+1
  return return_values


In [6]:
### b) und d)

data_1 = [(2,8), (2,4), (3,2), (3,5), (5,4), (6,9), (9,6)]
data_2 = [(5,2), (4,6), (6,4), (6,6), (6,7), (7,4), (7,7)]
obj    = [(5,5)]

In [7]:
import pandas as pd

df_1 = pd.DataFrame(data_1, columns=["x", "y"])
df_2 = pd.DataFrame(data_2, columns=["x", "y"])
df_obj = pd.DataFrame(obj, columns=["x", "y"])

In [8]:
# Funktion zum Zeichnen der Daten

import plotly.graph_objects as go

def show_k_nearest(k: int, manhatten=True):
  if manhatten:
    distance = get_k_nearest_dist(k, data_1+data_2, obj[0], manhattan_distance)
  else:
    distance = get_k_nearest_dist(k, data_1+data_2, obj[0], euler_distance)

  fig = go.Figure()

  fig.add_shape(type="circle",
                xref="x",
                yref="y",
                line_color="PaleTurquoise",
                fillcolor="LightSeaGreen",
                x0=obj[0][0]+distance,
                y0=obj[0][1]+distance,
                x1=obj[0][0]-distance,
                y1=obj[0][1]-distance, 
                opacity=0.3)

  fig.add_trace(go.Scatter(
      x=df_1["x"],
      y=df_1["y"],
      mode="markers",
      marker=dict(
          size=14,
          color="Black",
          line=dict(width=2,
                    color="Black")
          ),
      name="1. Klasse",
  ))

  fig.add_trace(go.Scatter(
      x=df_2["x"],
      y=df_2["y"],
      mode="markers",
      marker=dict(
          size=14,
          color="Green",
          line=dict(width=2,
                    color="Black")
          ),
      name="2. Klasse",
  ))

  fig.add_trace(go.Scatter(
      x=df_obj["x"],
      y=df_obj["y"],
      mode="markers",
      marker=dict(
          size=14,
          color="Red",
          line=dict(width=2,
                    color="Black")
          ),
      name="Unbekannte Klasse"
  ))

  fig.update_layout(
      autosize=False,
      width=1200,
      height=800,)

  fig.update_yaxes(
      scaleanchor = "x",
      scaleratio = 1,
      tickmode="linear",
      tick0=0,
      dtick=1,)

  fig.update_xaxes(
      tickmode="linear",
      tick0=0,
      dtick=1,)


  fig.show()

In [9]:
print(get_k_nearest(1, {"Klasse_1": data_1, "Klasse_2": data_2}, obj[0], manhattan_distance))
show_k_nearest(1, manhatten=True)

{'Klasse_1': [((5, 4), 1)]}


In [10]:
print(get_k_nearest(3, {"Klasse_1": data_1, "Klasse_2": data_2}, obj[0], manhattan_distance))
show_k_nearest(3, manhatten=True)

{'Klasse_1': [((5, 4), 1), ((3, 5), 2)], 'Klasse_2': [((4, 6), 2), ((6, 4), 2), ((6, 6), 2)]}


In [11]:
get_k_nearest(5, {"Klasse_1": data_1, "Klasse_2": data_2}, obj[0], manhattan_distance)
show_k_nearest(5, manhatten=True)

In [ ]:
### c)

# Einkommen ist als Zeichenkette angegeben, und sollte als numerischen Wert 
# angegeben werden.

In [12]:
### e) nominale Daten?

# -> Entweder Label Encoding (nur für nominal)
# -> oder One Hot Encoding, ist m.E. bei jedem Verfahren geeignet (für nominal und ordinal geeignet)

In [13]:
### f) Verbesserung des k-nearest Neighbour

# mehrere k-neighbour nehmen, und bsp. mittelwert nehmen
# andere Distanzfunktion verwenden, bsp. Eulersche Distanz